In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
pd.set_option('display.max_columns', 50)

In [2]:
pr_quot=pd.read_excel(r'C:\Users\Andrey\bsv\2\data\КС сокращенный.xlsx')

In [8]:
pr_quot.head()

,Unnamed: 0,Идентификатор,Начальная стоимость,Итоговоя стоимость,Дата начала,Количество снижений,Количество участников КС,Номер оферты по итогам,Статус контракта,Реестровый номер контракта,Номер контракта,Идентификатор СТЕ,Количество товара,Начальная цена за единицу,Конечная цена,Наименование товара,Вид,region,percent,day_of_week,week
0,0,378292,3544.36,2994.73,2018-10-19,31,2,60349361,Заключен,18-2075782,2053-172-2018,1203443,1.0,3544.36,2994.73,"Кресло оператора BRABIX ""Saturn ER-400"", с под...",Товар,77,15.507172,4,42
1,1,377037,8925.00,7245.00,2018-10-19,35,3,60319590,Заключен,18-2071382,536/18,1209487,300.0,15.32,12.44,Оптические диски,Товар,77,18.823529,4,42
2,2,377037,8925.00,7245.00,2018-10-19,35,3,60319589,Заключен,18-2071382,536/18,1209486,300.0,14.43,11.71,Оптические диски,Товар,77,18.823529,4,42
3,3,380149,11662.30,7310.80,2018-10-22,30,6,60404072,Заключен,18-2075817,18-2075817,1207700,15.0,96.80,60.68,"Мешки для мусора 120 л, черные, в рулоне 10 шт...",Товар,77,37.312537,0,43
4,4,380149,11662.30,7310.80,2018-10-22,30,6,60404070,Заключен,18-2075817,18-2075817,1207497,20.0,46.79,29.33,"Мешки для мусора 30 л, черные, в рулоне 30 шт....",Товар,77,37.312537,0,43


In [6]:
offer_data=pd.read_excel(r'C:\Users\Andrey\bsv\2\data\х Оферты.xlsx')
offer2_data=pd.read_excel(r'C:\Users\Andrey\bsv\2\data\1я половина новых оферт.xlsx')
offers=offer_data.append(offer2_data,ignore_index=True)

In [7]:
offers.head()

,Номер оферты,Регион поставщика,ИНН поставщика,КПП поставщика,Дата начала действия оферты,Дата окончания действия оферты,Идентификатор СТЕ,Стимость оферты,Вид продукции по классификатору портала,Адрес регистрации поставщика,ОКПД2,Регионы поставки
0,4198423-19,г Москва,7726651037,772701001.0,2019-10-21 00:00:00.000,2020-01-21 00:00:00.000,16548843,99990.00,Техническое обслуживание и текущий ремонт проч...,"г. Москва, улица Адмирала Руднева, дом 4, этаж...",NaN,г Москва; обл Московская;
1,4198343-19,г Москва,7726651037,772701001.0,2019-10-21 00:00:00.000,2020-01-21 00:00:00.000,16548180,99990.00,Техническое обслуживание и текущий ремонт проч...,"г. Москва, улица Адмирала Руднева, дом 4, этаж...",NaN,г Москва; обл Московская;
2,4851344-19,обл Московская,5029140070,502901001.0,2019-12-23 00:00:00.000,2020-03-23 00:00:00.000,1402614,10400.00,Техническое обслуживание прочего оборудования,"141033, Россия, Московская обл., г. Мытищи, у...",NaN,г Москва; обл Московская;
3,4375272-19,обл Московская,5029140070,502901001.0,2019-11-11 00:00:00.000,2020-02-11 00:00:00.000,1392730,13200.00,Техническое обслуживание и текущий ремонт проч...,"141033, Россия, Московская обл., г. Мытищи, у...",NaN,г Москва; обл Московская;
4,4308629-19,обл Самарская,631902289998,NaN,2019-10-31 00:00:00.000,2020-01-31 00:00:00.000,20133087,132.76,Крученая нерассасывающаяся полиамидная нить (э...,"г. Самара, ул. Ново-Садовая, д. 365, кв. 19.",NaN,г Москва; обл Московская;


In [28]:
def get_existing_offers(offs,id_ste,date):
    date_treshold=date + dt.timedelta(-14)
    return offs[(offs['Идентификатор СТЕ']==id_ste) \
                & (pd.DatetimeIndex(offs['Дата начала действия оферты'])<=date) \
               & (pd.DatetimeIndex(offs['Дата окончания действия оферты'])>=date_treshold)]

def get_existing_offers_count(offs,id_ste,date):
    return get_rows_count(get_existing_offers(offs,id_ste,date))

def get_rows_count(df):
    return df.shape[0]

In [65]:
offer_counts=offers.pivot_table(index='Идентификатор СТЕ',values='Номер оферты', aggfunc='count').reset_index() \
    .rename(columns={'Номер оферты':'offers_count'})

In [66]:
pr_quot.merge(offer_counts,on='Идентификатор СТЕ',how='left')

,Unnamed: 0,Идентификатор,Начальная стоимость,Итоговоя стоимость,Дата начала,Количество снижений,Количество участников КС,Номер оферты по итогам,Статус контракта,Реестровый номер контракта,Номер контракта,Идентификатор СТЕ,Количество товара,Начальная цена за единицу,Конечная цена,Наименование товара,Вид,region,percent,day_of_week,week,offers_count
0,0,378292,3544.36,2994.73,2018-10-19,31,2,60349361,Заключен,18-2075782,2053-172-2018,1203443,1.0,3544.36,2994.73,"Кресло оператора BRABIX ""Saturn ER-400"", с под...",Товар,77,15.507172,4,42,1.0
1,1,377037,8925.00,7245.00,2018-10-19,35,3,60319590,Заключен,18-2071382,536/18,1209487,300.0,15.32,12.44,Оптические диски,Товар,77,18.823529,4,42,1.0
2,2,377037,8925.00,7245.00,2018-10-19,35,3,60319589,Заключен,18-2071382,536/18,1209486,300.0,14.43,11.71,Оптические диски,Товар,77,18.823529,4,42,4.0
3,3,380149,11662.30,7310.80,2018-10-22,30,6,60404072,Заключен,18-2075817,18-2075817,1207700,15.0,96.80,60.68,"Мешки для мусора 120 л, черные, в рулоне 10 шт...",Товар,77,37.312537,0,43,1.0
4,4,380149,11662.30,7310.80,2018-10-22,30,6,60404070,Заключен,18-2075817,18-2075817,1207497,20.0,46.79,29.33,"Мешки для мусора 30 л, черные, в рулоне 30 шт....",Товар,77,37.312537,0,43,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251120,251120,686451,31577.50,21471.90,2020-01-22,64,9,798668565,Заключен,20-3224401,878/9-2020,20014727,5.0,2495.50,1696.78,Контроллер с встроенным считывателем Matrix-II...,Товар,77,32.002533,2,4,3.0
251121,251121,686451,31577.50,21471.90,2020-01-22,64,9,798668563,Заключен,20-3224401,878/9-2020,19968960,3.0,4200.00,2856.00,Цветной видеодмофон FE-4CHP2,Товар,77,32.002533,2,4,1.0
251122,251122,686451,31577.50,21471.90,2020-01-22,64,9,798668564,Заключен,20-3224401,878/9-2020,17999524,10.0,650.00,442.00,Z-5R Автономный контроллер СКУД IronLogic,Товар,77,32.002533,2,4,1.0
251123,251123,686452,30400.00,16872.00,2020-01-22,89,10,798421403,Заключен,20-3223498,24/44/20,18989088,200.0,152.00,84.36,Набор кистей,Товар,77,44.500000,2,4,4.0


In [68]:
pr_quot.percent.describe()

count    251125.000000
mean         17.318068
std          17.440580
min           0.498481
25%           1.001657
50%          12.638784
75%          29.002282
max          99.999974
Name: percent, dtype: float64